In [1]:
import os
import random
import numpy as np
import pandas as pd
import math

from glob import glob
from IPython.display import display

In [199]:
#Load Validation dataset & pCTRs
Path = 'C://Datasets//Multiagent//calc'

#Importing pCTR data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*validation_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Importing Validation data.
os.chdir('c://Datasets//Multiagent//rtb')
Datasets_features = {}
for Filename in glob('Validation.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Validation = Datasets_features['Validation']

In [342]:
class Agent:
        
    def __init__(self, id, budget, strategy, params):
        self.budget = budget
        self.id = id
        self.strategy = strategy
        self.params = params
        self.pCTRs = list(Datasets_CTR[self.strategy +'_validation_pCTR']['pCTR'].values)
        self.bids = tuple()
     
    # Update budget based on bid, returns True if budget allows
    def updateBudget(self,bid):
        if (self.budget - bid) < 0:
            print("ERROR: not enough budget for bid")
            return False
        else:
            self.budget = self.budget - bid
            return True
        
    def canBid(self,bid):
        if(self.budget>bid):
            return True
        return False
    
    #Ensure that bid is within budget
    def getMaxBid(self,idx):
        if(self.budget > self.bids[idx]):
            return self.bids[idx]
        else:
            return self.budget
    
    def addBids(self,bids):
        self.bids = bids
        return
    

In [343]:
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

In [354]:
Agent1 = Agent(1,6250, "LR", [90])
Agent2 = Agent(2,6250, "LR", [80])
Agent3 = Agent(3,6250, "LR", [70])
Agent4 = Agent(4,6250, "LR", [100])
Agent5 = Agent(5,6250, "LR", [110])
Agent6 = Agent(6,6250, "LR", [120])
 
#AgentBid1 = [Agent1,1.5]
#AgentBid2 = [Agent2, 2]
#AgentBid3 = [Agent3, 3]
#AgentBid4 = [Agent4, 1.5]
#AgentBid5 = [Agent5, 4]
#AgentBid6 = [Agent6, 3]

initPCR = 0.0007375623256

getBidsForAgent(Agent1,initPCR)
getBidsForAgent(Agent2,initPCR)
getBidsForAgent(Agent3,initPCR)
getBidsForAgent(Agent4,initPCR)
getBidsForAgent(Agent5,initPCR)
getBidsForAgent(Agent6,initPCR)

Agents = [Agent1,Agent2,Agent3,Agent4,Agent5,Agent6]

#AgentsBids = []
#AgentsBids.append(AgentBid1)
#AgentsBids.append(AgentBid2)
#AgentsBids.append(AgentBid3)
#AgentsBids.append(AgentBid4)
#AgentsBids.append(AgentBid5)
#AgentsBids.append(AgentBid6)
debug = True


#df = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])

AgentsBidsDf = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])


def getBidsForAgent(agent,initPCR):    
    if(agent.strategy == "LR"):
        bids = []
        for pCTR in agent.pCTRs:
            bids.append(linear_bidding(agent.params[0],pCTR,initPCR))        
        agent.addBids(bids)
    else:
        print("Error: Strategy not found")
    return 

def getBidsCurrentAuction(agents, idx, verbose=False):
    bidsCurrentAuction = []
    for agent in agents:        
        if(verbose):
            print('Bid[',agent.id,']:', agent.getMaxBid(idx))
        bidsCurrentAuction.append([agent, agent.getMaxBid(idx)])
    return pd.DataFrame(bidsCurrentAuction, columns=["Agent","Bid"])

def secondPriceWinner(AgentsBidsDf, floorPrice,idx=0, verbose=False):
    bestBids= AgentsBidsDf.sort_values(by="Bid", ascending=False)
    winningAgent = bestBids.iloc[0,0]
    secondPrice = bestBids.iloc[1,1]
    if(secondPrice > floorPrice):
        winningAgent.updateBudget(secondPrice)
        if (verbose):
            print(idx,"\tWon: ", winningAgent.id, " Paid:" , '{:.1f}'.format(secondPrice), " Upd budget:", '{:.1f}'.format(winningAgent.budget))
    else:        
        if(verbose):
            print(idx, "\tNo winner, Floor:",floorPrice)    
    return winningAgent.id

def individualAuctions(verbose=False):
    #for idx in range(len(Validation)):   
    for idx in range(300):    
        #curr_bid = Validation.iloc[idx]
        #if(verbose):
        #    print (Validation.iloc[idx])
        df = getBidsCurrentAuction(Agents,idx,False)
        floorPrice = Validation['payprice'][idx]
        secondPriceWinner(df,floorPrice,idx,verbose)
    return
        


In [355]:
print(individualAuctions(True))


0 	Won:  6  Paid: 94.0  Upd budget: 6156.0
1 	No winner, Floor: 75
2 	No winner, Floor: 65
3 	Won:  6  Paid: 43.5  Upd budget: 6112.6
4 	Won:  6  Paid: 31.3  Upd budget: 6081.3
5 	Won:  6  Paid: 143.2  Upd budget: 5938.1
6 	Won:  6  Paid: 97.7  Upd budget: 5840.5
7 	Won:  6  Paid: 52.1  Upd budget: 5788.4
8 	No winner, Floor: 58
9 	No winner, Floor: 55
10 	Won:  6  Paid: 94.9  Upd budget: 5693.5
11 	No winner, Floor: 55
12 	No winner, Floor: 49
13 	No winner, Floor: 110
14 	No winner, Floor: 160
15 	Won:  6  Paid: 33.4  Upd budget: 5660.1
16 	No winner, Floor: 50
17 	No winner, Floor: 76
18 	No winner, Floor: 89
19 	No winner, Floor: 136
20 	Won:  6  Paid: 65.2  Upd budget: 5594.9
21 	Won:  6  Paid: 77.5  Upd budget: 5517.4
22 	No winner, Floor: 64
23 	No winner, Floor: 129
24 	Won:  6  Paid: 170.6  Upd budget: 5346.8
25 	No winner, Floor: 166
26 	No winner, Floor: 88
27 	No winner, Floor: 221
28 	No winner, Floor: 69
29 	No winner, Floor: 70
30 	No winner, Floor: 20
31 	Won:  6  Paid:

294 	No winner, Floor: 121
295 	No winner, Floor: 88
296 	No winner, Floor: 52
297 	Won:  4  Paid: 28.6  Upd budget: 4159.8
298 	No winner, Floor: 77
299 	No winner, Floor: 42
None


In [274]:
#individualAuctions()


print((df))
floorPrice = Validation['slotprice'][1]
secondPriceWinner(df)
#print(df)

Bid[ 1 ]: 117.15830729597856
Bid[ 2 ]: 104.14071759642539
Bid[ 3 ]: 91.12312789687222
Bid[ 4 ]: 130.17589699553173
Bid[ 5 ]: 143.1934866950849
                                           Agent         Bid
0  <__main__.Agent object at 0x00000218D4F89320>  117.158307
1  <__main__.Agent object at 0x00000218D4F89358>  104.140718
2  <__main__.Agent object at 0x00000218D4F893C8>   91.123128
3  <__main__.Agent object at 0x00000218B5A7D0F0>  130.175897
4  <__main__.Agent object at 0x00000218B5A7DBA8>  143.193487
Auction won by  5  Paid: 130.17589699553173  Upd budget:  5975.756487115378


5

In [250]:
#agent1.maxBid(6300)
#print(secondPriceWinner(AgentsBidsDf,4))
#print(createBid(Agent1),)
#print((Agent1.pCTRs))

    

print(Agent1.bids[0])
print(Agent2.bids[0])
print(Agent3.bids[0])
print(Agent4.bids[0])
print(Agent5.bids[0])


76.87248358372172
68.33109651886376
59.78970945400579
85.41387064857969
93.95525771343766


In [139]:
print(Agent5.budget)

6244.0


In [90]:
bestBids= df.sort_values(by="Bid", ascending=False)
bestBids

,Agent,Bid
4,<__main__.Agent object at 0x00000218B5A5D358>,6.0
3,<__main__.Agent object at 0x00000218B5A5D320>,5.0
2,<__main__.Agent object at 0x00000218B5A5D2E8>,3.0
5,<__main__.Agent object at 0x00000218B5A5D390>,3.0
1,<__main__.Agent object at 0x00000218B5A5D278>,2.0
0,<__main__.Agent object at 0x00000218B5A5D208>,1.5


In [123]:
print("best bid by: ", bestBids.iloc[0,0].id, " second " , bestBids.iloc[1,0].id)
print("Win price: ", bestBids.iloc[0,1], " Price to pay " , bestBids.iloc[1,1])


best bid by:  5  second  4
Win price:  6.0  Price to pay  5.0
